# Utilizando OpenStreetMap

Vamos a utilizar ahora OpenRoute service para mostrar un caso de uso real

In [18]:
#Importar librerias

import random
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cdist
from ortools.linear_solver import pywraplp
import time
import numpy as np
import folium
from folium.plugins import BeautifyIcon
import openrouteservice as ors
import osmnx as ox



In [19]:
# issue https://github.com/microsoft/vscode/issues/266193

from folium import Map
import base64
from IPython.display import IFrame, display

def show_folium_safe(m : Map, height=1000):
    """
    Displays a Folium map in a safe IFrame using Base64 encoding.
    This avoids "Trusted" errors, file path issues, and CSS leakage.
    """
    # 1. Get the raw HTML string of the map
    html_content = m.get_root().render()
    
    # 2. Encode the HTML to base64
    # This allows us to put the entire map "inside" the URL string
    encoded = base64.b64encode(html_content.encode('utf-8')).decode('utf-8')
    
    # 3. Create a Data URI
    data_uri = f"data:text/html;charset=utf-8;base64,{encoded}"
    
    # 4. Display the IFrame
    # We use width='100%' to fill the cell width, but the CSS is trapped inside
    display(IFrame(src=data_uri, width="100%", height=height))

In [20]:
# Mapa alrededor de Rosario

# Coordenadas del Monumento a la Bandera (aprox)
lat_monumento = -32.9478
lon_monumento = -60.6305

m = folium.Map(location=[lat_monumento, lon_monumento], zoom_start=15)

generar puntos aleatorios en rosario

In [21]:
print("Descargando red vial del Centro de Rosario...")

# Descargar calles a 1500 metros a la redonda del punto
G = ox.graph_from_point((lat_monumento, lon_monumento), dist=1500, network_type='drive')


# 2. Convertir el grafo a GeoDataFrames (nodos y aristas)
nodes, edges = ox.graph_to_gdfs(G)

# 3. Muestreo Aleatorio
# Seleccionamos 100 nodos (intersecciones) al azar de la red vial
# Esto asegura que los puntos existan físicamente en una calle
puntos_reales = nodes.sample(n=50, random_state=42)

# 4. Crear el DataFrame limpio para tu VRP
df_vrp = pd.DataFrame({
    'osmid': puntos_reales.index, # ID oficial de OpenStreetMap (útil para calcular distancias)
    'latitud': puntos_reales.y.values,
    'longitud': puntos_reales.x.values
})

print(f"Se generaron {len(df_vrp)} puntos válidos sobre calles.")
print(df_vrp.head())


Descargando red vial del Centro de Rosario...
Se generaron 50 puntos válidos sobre calles.
        osmid    latitud   longitud
0   257491698 -32.945130 -60.637867
1   257489792 -32.938708 -60.640588
2   257489020 -32.955139 -60.646171
3  3364478903 -32.952454 -60.627765
4   257490114 -32.943646 -60.640368


In [22]:
for idx, row in df_vrp.iterrows():
    folium.Marker(
        location = [row['latitud'], row['longitud']],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=idx,
            border_color='blue',
            text_color='white',
            background_color='blue'
        )
    ).add_to(m)

In [23]:
show_folium_safe(m)

In [24]:
import networkx as nx
import osmnx as ox
import pandas as pd
import numpy as np

G = ox.add_edge_speeds(G) 
G = ox.add_edge_travel_times(G)

In [25]:
# Lista de los IDs de los nodos que vamos a usar
nodos_interes = df_vrp['osmid'].tolist()
n = len(nodos_interes)

# Inicializamos matrices vacías con ceros
matriz_distancia = np.zeros((n, n))
matriz_tiempo = np.zeros((n, n))

print(f"Calculando rutas entre {n} puntos... (esto puede tomar un momento)")

# Iteramos sobre cada origen y destino
for i, origen in enumerate(nodos_interes):
    for j, destino in enumerate(nodos_interes):
        if i == j:
            continue # Distancia a sí mismo es 0
        
        try:
            # Calcular distancia en metros (weight='length')
            dist = nx.shortest_path_length(G, source=origen, target=destino, weight='length')
            matriz_distancia[i][j] = dist
            
            # Calcular tiempo en segundos (weight='travel_time')
            tiempo = nx.shortest_path_length(G, source=origen, target=destino, weight='travel_time')
            matriz_tiempo[i][j] = tiempo
            
        except nx.NetworkXNoPath:
            # Si no hay camino (ej: calle cortada o sentido único imposible), ponemos un valor muy alto
            matriz_distancia[i][j] = 999999
            matriz_tiempo[i][j] = 999999

print("Cálculo terminado.")

Calculando rutas entre 50 puntos... (esto puede tomar un momento)
Cálculo terminado.


In [26]:
ids = df_vrp['osmid'].values

# dfs de distancias y tiempos
df_matrix_dist = pd.DataFrame(matriz_distancia, index=ids, columns=ids)
df_matrix_time = pd.DataFrame(matriz_tiempo, index=ids, columns=ids)

print(df_matrix_dist.iloc[:5, :5])
print(df_matrix_time.iloc[:5, :5])

             257491698    257489792    257489020    3364478903   257490114 
257491698      0.000000  1039.159428  1780.353610  1912.300513   639.883679
257489792   1030.066008     0.000000  2292.284526  2444.739573   665.809449
257489020   1986.606401  2028.515489     0.000000  2242.456727  1629.239740
3364478903  1585.061390  2386.660940  2277.549876     0.000000  1991.587555
257490114    364.256559   887.023520  1629.551591  2145.402615     0.000000
            257491698   257489792   257489020   3364478903  257490114 
257491698     0.000000  100.908349  172.287340  185.435274   65.105353
257489792    95.758976    0.000000  206.305607  178.581317   59.922850
257489020   190.986132  191.084824    0.000000  180.256413  155.150006
3364478903  144.769905  175.022581  204.979489    0.000000  156.326987
257490114    35.836125   83.091077  154.031031  179.578271    0.000000


In [27]:
df_matrix_dist

,257491698,257489792,257489020,3364478903,257490114,257491537,257490219,257490605,257490911,257490685,...,1650518379,282227350,257489256,1307963066,257490570,282227139,257490263,257490113,257489001,257490000
257491698,0.000000,1039.159428,1780.353610,1912.300513,639.883679,2834.940086,1115.974646,2013.032716,1926.982715,724.299325,...,891.113485,1388.713098,1372.385848,961.541336,894.740808,791.075281,375.654034,508.099585,896.355792,1247.156665
257489792,1030.066008,0.000000,2292.284526,2444.739573,665.809449,3367.379147,1630.392853,2863.275182,2461.124004,1754.365333,...,848.856426,2418.779106,871.367194,691.467330,1300.459440,1301.332531,922.330574,532.481861,930.668513,2057.019195
257489020,1986.606401,2028.515489,0.000000,2242.456727,1629.239740,2126.582594,917.201574,1296.940524,1716.726714,1603.094151,...,2397.459278,672.655961,2361.741909,2467.887129,2296.695381,1780.431342,1881.999827,2014.445378,1885.711853,2125.951491
3364478903,1585.061390,2386.660940,2277.549876,0.000000,1991.587555,1464.256437,1637.592839,1751.656156,558.001294,946.050960,...,1308.470591,1873.453093,2719.887359,1453.178457,1153.758630,2376.136671,1725.814416,1858.259967,2256.446619,430.118252
257490114,364.256559,887.023520,1629.551591,2145.402615,0.000000,3068.042189,964.583404,2341.552500,2161.787046,1088.555884,...,775.109436,1752.969657,1220.249940,845.537286,778.736759,638.939373,259.649984,392.095535,744.219884,1535.296514
257491537,2363.274435,3164.873984,2123.123514,778.213044,2769.800599,0.000000,2000.659359,1586.775456,920.947095,1724.264004,...,2086.683636,1708.572393,3498.100404,2231.391501,1931.971674,3154.349716,2504.027460,2636.473011,3034.659663,1208.331297
257490219,1069.404828,1622.967085,1195.262569,1327.209472,1223.691336,2249.849045,0.000000,1427.941675,1341.891674,687.846896,...,1480.257704,803.622057,1956.193505,1550.685555,1381.448125,1374.882938,964.798253,1097.243804,1480.163449,1210.704235
257490605,2236.369083,3263.961082,1836.039505,1735.597876,2867.352635,1613.951055,1713.575350,0.000000,1209.163765,1585.704785,...,2216.263309,943.493881,3597.187501,2360.971175,1805.066323,3027.444364,2601.579496,2734.025046,3132.211699,1626.201011
257490911,1582.199627,2609.791626,1719.548582,526.434110,2213.183178,1449.073684,1083.387761,1193.654862,0.000000,935.237590,...,1562.093853,1315.451799,2943.018045,1706.801719,1150.896866,2373.274908,1947.410039,2079.855590,2478.042242,419.304883
257490685,684.141669,1711.733667,1847.406743,1190.375405,1315.125220,2113.014979,1203.968203,1321.513023,1206.759836,0.000000,...,820.348633,1443.309960,2044.960087,965.056499,693.601230,1475.216950,1049.352081,1181.797632,1579.984284,522.857340


Ahora ya tenemos las matrices para trabajar el TSP

In [28]:
def points_sub(points, i):
    """
    De una lista de numeros sustrae el numero i en una copia nueva.
    Se considera que poinla lista no tiene elementos duplicados (por ser lista de indices) y que i se encuentra en la lista.

    Args:
      points: lista de puntos.
      i: elemento a sustraer de la lista

    Returns:
      new: lista nueva similar a points pero sin el elemento i.
    """
    new = points.copy()
    new.remove(i)

    return new

In [29]:
distancias = df_matrix_dist  #Cambiar si queremos usar tiempos

# Definimos el modelo
modelo_mtz = pywraplp.Solver.CreateSolver('SAT')

list_index_puntos = list(distancias.index)

# Definimos como punto de partida el index 0
partida = list_index_puntos[0]

# uso n como n-1 ya que arranco de indice 0 en la lista. Por esto en mtz no resto 1 ni en u_i
n = max(list_index_puntos)

# Definimos variables

x_ij={i:{j: modelo_mtz.IntVar(0,1,'x_'+str(i)+'_'+str(j)) for j in points_sub(list_index_puntos, i)} for i in list_index_puntos}
u_i = {i: modelo_mtz.IntVar(1,n,'u_'+str(i)) for i in points_sub(list_index_puntos, partida)}

# Definimos la función objetivo
obj_expr = sum(x_ij[i][j] * distancias[i][j] for i in list_index_puntos for j in points_sub(list_index_puntos, i))


#Restricciones:

#1
for j in list_index_puntos:
    modelo_mtz.Add(sum(x_ij[i][j] for i in points_sub(list_index_puntos, j)) == 1)

#2
for i in list_index_puntos:
    modelo_mtz.Add(sum(x_ij[i][j] for j in points_sub(list_index_puntos, i)) == 1)

#3
for i in points_sub(list_index_puntos, partida):
    for j in points_sub(points_sub(list_index_puntos, partida), i):
        modelo_mtz.Add(u_i[i] - u_i[j] + 1 <= n * (1- x_ij[i][j]))


# Solver
modelo_mtz.Minimize(obj_expr)

In [30]:
inicio = time.time()
status = modelo_mtz.Solve()
fin = time.time()
print(f"Tiempo de resolución: {fin - inicio} segundos")

Tiempo de resolución: 7.927496433258057 segundos


In [31]:
modelo_mtz.Objective().Value()

1019847.2198994895

Visualización

In [15]:
# Conectar en el mapa 'm' los arcos seleccionados por modelo_mtz (x_ij).
# Muestra también la ruta ordenada empezando en 'partida' si se puede reconstruir.

# Usamos df_vrp (osmid, latitud, longitud), x_ij, partida, status y show_folium_safe ya definidos.

df_idx = df_vrp.set_index('osmid')

# 1) Dibujar todas las aristas seleccionadas (valor 1)
for i, tos in x_ij.items():
    for j, var in tos.items():
        try:
            if var.solution_value() > 0.5:
                try:
                    a = df_idx.loc[i]
                    b = df_idx.loc[j]
                except KeyError:
                    # intentar conversión por si los tipos difieren
                    a = df_idx.loc[int(i)]
                    b = df_idx.loc[int(j)]
                folium.PolyLine(
                    locations=[[a['latitud'], a['longitud']], [b['latitud'], b['longitud']]],
                    color='red', weight=2, opacity=0.8
                ).add_to(m)
        except Exception:
            # si no hay solución o var no tiene solution_value, ignorar
            pass

# 2) Intentar reconstruir el tour (secuencia) empezando en 'partida'
tour = []
if status == pywraplp.Solver.OPTIMAL or status == 0:
    cur = partida
    tour = [cur]
    visited = set(tour)
    for _ in range(len(df_idx)):
        # buscar siguiente desde cur
        next_node = None
        for j, var in x_ij.get(cur, {}).items():
            try:
                if var.solution_value() > 0.5:
                    next_node = j
                    break
            except Exception:
                continue
        if next_node is None:
            break
        tour.append(next_node)
        if next_node == partida:
            break
        if next_node in visited:
            break
        visited.add(next_node)
        cur = next_node

# 3) Dibujar la polilínea del tour ordenado (azul, más gruesa)
if len(tour) > 1:
    coords = []
    for node in tour:
        try:
            r = df_idx.loc[node]
        except KeyError:
            r = df_idx.loc[int(node)]
        coords.append([r['latitud'], r['longitud']])
    folium.PolyLine(locations=coords, color='blue', weight=4, opacity=0.9).add_to(m)

# Mostrar el mapa
show_folium_safe(m)

In [ ]:
# Visualización del recorrido óptimo usando rutas reales de calles (no líneas rectas)
# Usamos el tour calculado en la celda anterior y el grafo G de OSMnx

if len(tour) > 1:
    for i in range(len(tour) - 1):
        origen = tour[i]
        destino = tour[i + 1]
        try:
            # Obtener el camino real en la red vial
            path = nx.shortest_path(G, origen, destino, weight='length')
            # Extraer coordenadas de cada nodo del camino
            coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in path]
            folium.PolyLine(locations=coords, color='green', weight=3, opacity=0.7).add_to(m)
        except Exception as e:
            print(f"No se pudo trazar de {origen} a {destino}: {e}")

show_folium_safe(m)

No se pudo trazar de 257491537 a 257491184: No path between 257491537 and 257491184.


: 